In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import config

In [2]:
#Load google.cloud.bigquery
%load_ext google.cloud.bigquery

In [3]:
#Select path to credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.GOOGLE_APPLICATION_CREDENTIALS

In [4]:
%%bigquery --use_rest_api ZRI_MF
SELECT *
FROM `high-empire-220313.ZRI.Multi_Family`

In [5]:
%%bigquery --use_rest_api Zip_5yr
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr` 

In [6]:
from data_setup import ZRI_format

In [7]:
%%time
ZRI_new = ZRI_format(ZRI_MF, time_unit = 'Month', window_size = 3, future_time = 4)

Wall time: 42 s


In [8]:
#Columns to use in the final analysis
zip_columns = ['geo_id','unemployed_pop','white_pop','vacant_housing_units','total_pop','worked_at_home',
               'poverty','percent_income_spent_on_rent','occupied_housing_units',
               'median_year_structure_built','median_age','married_households','masters_degree',
              'male_pop','female_pop','income_per_capita','housing_units','employed_pop','black_pop',
              'asian_pop','amerindian_pop','graduate_professional_degree']

In [12]:
#Merge zip code data onto the ZRI data
final_data = ZRI_new.merge(Zip_5yr[zip_columns],how = 'left',left_on='ZipCode',right_on ='geo_id')

#Convert columns to percentage
#Columns to divide by total population
pop_columns = ['unemployed_pop','white_pop','masters_degree',
               'graduate_professional_degree','employed_pop','black_pop',
              'asian_pop','amerindian_pop','poverty','worked_at_home']

#Columns to divide by total housing units
house_columns = ['vacant_housing_units','occupied_housing_units']

#Division
final_data.loc[:,pop_columns] = static_data[pop_columns].div(static_data['total_pop'], axis = 0)
final_data.loc[:,house_columns] = static_data[house_columns].div(static_data['housing_units'], axis = 0)

In [13]:
final_data

,Target_index,Target_ZRI,Month,Year,ZRI_minus_4M,ZRI_minus_5M,ZRI_minus_6M,ZipCode,geo_id,unemployed_pop,...,masters_degree,male_pop,female_pop,income_per_capita,housing_units,employed_pop,black_pop,asian_pop,amerindian_pop,graduate_professional_degree
0,10201001013,996.0,10,2010,NaN,NaN,NaN,01013,01013,1.550769e-06,...,1.738741e-06,10958.0,12107.0,23656.0,10192.0,0.000020,1.616559e-06,9.924920e-07,1.503776e-08,2.233107e-06
1,10201001020,1058.0,10,2010,NaN,NaN,NaN,01020,01020,1.107273e-06,...,1.122728e-06,14803.0,15294.0,30280.0,13326.0,0.000017,1.070842e-06,6.734160e-07,6.623764e-08,1.458332e-06
2,10201001040,1025.0,10,2010,NaN,NaN,NaN,01040,01040,9.501777e-07,...,1.205358e-06,19780.0,20596.0,23385.0,16855.0,0.000010,8.109328e-07,3.042532e-07,5.766088e-08,1.515745e-06
3,10201001085,NaN,10,2010,NaN,NaN,NaN,01085,01085,6.952295e-07,...,1.359194e-06,20243.0,21699.0,31563.0,16341.0,0.000012,4.019029e-07,6.872710e-07,2.160157e-08,1.797478e-06
4,10201001089,NaN,10,2010,NaN,NaN,NaN,01089,01089,1.197460e-06,...,2.711321e-06,14224.0,14442.0,31135.0,12463.0,0.000018,9.139149e-07,1.606348e-06,7.423277e-08,3.348993e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210288,9201999501,1246.0,9,2019,1222.0,1219.0,1225.0,99501,99501,2.110144e-06,...,3.398913e-06,8872.0,8186.0,42060.0,8717.0,0.000029,4.625821e-06,3.996902e-06,6.306376e-06,5.828672e-06
210289,9201999504,1417.0,9,2019,1423.0,1419.0,1416.0,99504,99504,7.063030e-07,...,1.123110e-06,22586.0,19918.0,33308.0,15959.0,0.000012,2.203045e-06,2.446598e-06,1.668336e-06,1.485672e-06
210290,9201999508,1282.0,9,2019,1265.0,1258.0,1255.0,99508,99508,1.059386e-06,...,1.299187e-06,17107.0,18498.0,30856.0,13659.0,0.000014,1.955485e-06,3.426635e-06,3.664070e-06,1.920777e-06
210291,9201999654,1271.0,9,2019,1296.0,1307.0,1318.0,99654,99654,6.993445e-07,...,5.102625e-07,32691.0,29444.0,31043.0,20893.0,0.000007,1.577410e-07,2.688591e-07,8.721085e-07,6.659314e-07


In [14]:
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [15]:
#Imputation Strategy
#Dropna for now
final_data = final_data.dropna()

In [34]:
#Find feature columns
full_feature_columns = [x for x in final_data.columns if 'minus' in x] +\
                                                     pop_columns  +\
                                                    house_columns +\
                                                    ['income_per_capita',
                                                    'percent_income_spent_on_rent',
                                                    'median_age']

In [35]:
ZRI_feature_columns = [x for x in final_data.columns if 'minus' in x]

In [36]:
#Train test split, test data is above a given year
test_year = 2019
training_data = final_data[final_data.Year < test_year]
final_test_data = final_data[final_data.Year >= test_year]

In [37]:
 X_train, X_test, y_train, y_test = train_test_split(training_data[feature_columns],
                                                     training_data['Target_ZRI'],
                                                     test_size = .1,
                                                     random_state = 42
                                                    ) 

In [38]:
 X_train_zri, X_test_zri, y_train_zri, y_test_zri = train_test_split(training_data[ZRI_feature_columns],
                                                     training_data['Target_ZRI'],
                                                     test_size = .1,
                                                     random_state = 42
                                                    ) 

In [39]:
lr_zri = LinearRegression()
lr_full = LinearRegression()

In [40]:
lr_full.fit(X_train,y_train); lr_zri.fit(X_train_zri, y_train_zri)

LinearRegression()

In [42]:
lr_zri.score(X_test_zri,y_test_zri) , lr_zri.score(X_train_zri, y_train_zri)

(0.9940922374377177, 0.9939800538960931)

In [43]:
lr_full.score(X_test,y_test), lr_full.score(X_train, y_train)

(0.9941262147435765, 0.9940299763870987)

In [44]:
prediction_error_full = final_test_data['Target_ZRI'] - lr_full.predict(final_test_data[full_feature_columns])

In [45]:
prediction_error_ZRI = final_test_data['Target_ZRI'] - lr_zri.predict(final_test_data[ZRI_feature_columns])

In [47]:
prediction_error_full.describe(), prediction_error_ZRI.describe()

(count    18034.000000
 mean        -6.103928
 std         50.272472
 min       -378.155522
 25%        -31.221562
 50%         -6.170689
 75%         20.387144
 max        332.982309
 Name: Target_ZRI, dtype: float64,
 count    18034.000000
 mean        -6.988601
 std         50.142288
 min       -382.748170
 25%        -31.869321
 50%         -6.831950
 75%         19.475290
 max        311.215308
 Name: Target_ZRI, dtype: float64)

In [ ]:
prediction_error = final_test_data['Target_ZRI'] - lr.predict(final_test_data[feature_columns])

In [ ]:
prediction_error.describe()

In [ ]:
lr.coef_, lr.alpha_

In [ ]:
window_sizes = list(range(1,13))
future_time = 24
time_unit = 'Month'
num_obs = defaultdict()
errors = defaultdict()
scores = defaultdict()
coefficients = defaultdict()

for window_size in window_sizes:
    ZRI_new = ZRI_format(ZRI_MF, time_unit = time_unit, 
                         window_size = window_size,
                         future_time = future_time)
    ZRI_new = ZRI_new.dropna()
    num_obs[window_size] = ZRI_new.shape[0]
    feature_columns = [x for x in ZRI_new.columns if 'minus' in x]
    test_year = 2019
    training_data = ZRI_new[ZRI_new.Year < test_year]
    final_test_data = ZRI_new[ZRI_new.Year >= test_year]
    most_recent_feature = f'ZRI_minus_{future_time}{time_unit[0]}'
    X_train, X_test, y_train, y_test = train_test_split(training_data[feature_columns],
                                                     training_data['Target_ZRI'],
                                                     test_size = .1
                                                    ) 
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    coefficients[window_size] = defaultdict()
    scores[window_size] = (lr.score(X_test,y_test), lr.score(X_train, y_train))
    errors[window_size] = (final_test_data['Target_ZRI'] - 
                           lr.predict(final_test_data[feature_columns])).div(final_test_data[most_recent_feature])
    

In [ ]:
lr.coef_

In [ ]:
plt.boxplot(errors.values())

In [ ]:
plt.boxplot(list(map(lambda x: x.apply(lambda y: np.log10(y+1250)),errors.values())))

In [ ]:
pd.DataFrame(errors).describe()